## --- Model MobileNETv2 --- 

In [ ]:
# Import framework
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Conv2D, MaxPooling2D, Concatenate, Add, Activation, Multiply, Input, UpSampling2D, GlobalMaxPooling2D , Add, BatchNormalization, DepthwiseConv2D
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import os
import numpy as np


In [ ]:
# coumt the number of images in the training folder
count = 0 
dirs = os.listdir('Fruits Data 20/val/')
for dir in dirs:
    files = list(os.listdir('Fruits Data 20/val/'+dir))
    print( dir +' Folder has '+ str(len(files)) + ' Images')
    count = count + len(files)
print( 'Images Folder has '+ str(count) + ' Images')

In [ ]:
base_train = 'Fruits Data 20/train/'
base_val = 'Fruits Data 20/val/'
IMG_SIZE = 224  
BATCH_SIZE = 32
EPOCHS = 10

### Load preprocessing

In [ ]:

def preprocess_train(image, label):
    # resize 
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    
    # Augmentation
    image = tf.image.random_brightness(image, 0.1)  
    image = tf.image.random_flip_left_right(image)  
    image = tf.image.random_flip_up_down(image)     
    image = tf.image.rot90(image, k=tf.random.uniform([], 0, 4, dtype=tf.int32))  # Xoay ngẫu nhiên 0-270 độ
    
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))  
    
    image = preprocess_input(image)
    
    return image, label

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    base_train,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    label_mode='categorical',
    seed=42
).shuffle(1000).map(preprocess_train, num_parallel_calls=tf.data.AUTOTUNE)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    base_val,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    label_mode='categorical',
    seed=42
).map(lambda image, label: (preprocess_input(image), label),  # Chỉ chuẩn hóa [-1, 1]
      num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)


### Show image after preprocessing

In [ ]:

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow((images[i] + 1) / 2)
        plt.title(labels[i])
        plt.axis("off")
plt.show()

### Res-Inception Module

In [ ]:
def res_inception_module(inputs, filters_1x1, filters_3x3_reduce, filters_3x3,
                        filters_5x5_reduce, filters_5x5, filters_pool):
    # Nhánh 1: Convolution 1x1
    branch_1x1 = Conv2D(filters_1x1, (1, 1), padding='same', activation='relu')(inputs) 

    # Nhánh 2: 1x1 giảm kênh + 3x3
    branch_3x3 = Conv2D(filters_3x3_reduce, (1, 1), padding='same', activation='relu')(inputs) 
    branch_3x3 = Conv2D(filters_3x3, (3, 3), padding='same', activation='relu')(branch_3x3)

    # Nhánh 3: 1x1 giảm kênh + 5x5
    branch_5x5 = Conv2D(filters_5x5_reduce, (1, 1), padding='same', activation='relu')(inputs)
    branch_5x5 = Conv2D(filters_5x5, (5, 5), padding='same', activation='relu')(branch_5x5)

    # Nhánh 4: Pooling + 1x1
    branch_pool = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(inputs)
    branch_pool = Conv2D(filters_pool, (1, 1), padding='same', activation='relu')(branch_pool)

    # Nối các nhánh
    inception_output = Concatenate(axis=-1)([branch_1x1, branch_3x3, branch_5x5, branch_pool])

    # Residual connection
    if inputs.shape[-1] != inception_output.shape[-1]:
        shortcut = Conv2D(inception_output.shape[-1], (1, 1), padding='same')(inputs)
    else:
        shortcut = inputs

    outputs = Add()([shortcut, inception_output])
    outputs = Activation('relu')(outputs)
    return outputs

### Efficient Multi-scale Attention (EMA) Module

In [ ]:
def ema_module(inputs, filters):
    # Multi-scale branches
    conv_1x1 = Conv2D(filters // 2, (1, 1), padding='same', activation='relu')(inputs)
    conv_3x3 = Conv2D(filters // 2, (3, 3), padding='same', activation='relu')(inputs)
    # Combined pooling: Global Average Pooling và Global Max Pooling
    global_avg_pool = GlobalAveragePooling2D(keepdims=True)(inputs) # tính trung bình tất cả các pixel trong vùng 
    global_max_pool = GlobalMaxPooling2D(keepdims=True)(inputs) # giữ lại đặc trưng nổi bậc nhất, giảm overfitting < mất thông tin về vùng không có giá trị lớn 

    # combine pooling
    global_pool = Add()([global_avg_pool, global_max_pool])  
    global_pool = Conv2D(filters // 2, (1, 1), padding='same', activation='relu')(global_pool)
    global_pool = UpSampling2D(size=(inputs.shape[1], inputs.shape[2]))(global_pool)

    # Concatenate multi-scale features
    multi_scale = Concatenate()([conv_1x1, conv_3x3, global_pool])

    # Attention weights
    attention = Conv2D(filters, (1, 1), padding='same')(multi_scale)
    attention = Activation('sigmoid')(attention)

    # Apply attention
    outputs = Multiply()([inputs, attention])
    return outputs


In [ ]:
def relu6(x):
    return Activation('relu6')(x) 

def bottleneck_block(inputs, filters, expansion, stride):
    """
    inputs: tensor đầu vào
    filters: số bộ lọc ở tầng pointwise cuối cùng
    expansion: hệ số mở rộng số kênh trong bottleneck
    stride: bước stride cho depthwise convolution
    """
    in_channels = inputs.shape[-1] 
    
    # 1. Expansion layer 
    x = Conv2D(filters * expansion, (1, 1), padding='same', use_bias=False)(inputs)
    x = BatchNormalization()(x)
    x = relu6(x)

    # 2. Depthwise Convolution
    x = DepthwiseConv2D((3, 3), strides=stride, padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = relu6(x)

    # 3. Projection layer 
    x = Conv2D(filters, (1, 1), padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    
    # Skip connection 
    if stride == 1 and in_channels == filters:
        x = Add()([inputs, x])
    
    return x

## Improve Model MobinetV2

### Load MobileNetV2 with pretrained

In [ ]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False

### Built Model MobileNetV2 with res_inception_module and ema_module

In [ ]:
# Xây dựng mô hình cải tiến
inputs = Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_model(inputs, training=False)  # MobileNetV2 backbone

# Res-Inception Module 
x = res_inception_module(x, 
                         filters_1x1=64, 
                         filters_3x3_reduce=96, filters_3x3=128,
                         filters_5x5_reduce=16, filters_5x5=32,
                         filters_pool=32) 

# EMA Module 
x = ema_module(x, filters=256)

x = GlobalAveragePooling2D()(x)  

x = Dense(64, activation='relu6')(x) 
x = Dropout(0.2)(x) # Dropout 20% to prevent overfitting
predictions = Dense(20, activation='softmax')(x) # 20 fruit classes

### Built Model MobileNetV2 with bottleneck_block

In [ ]:
# Xây dựng mô hình cải tiến
inputs = Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_model(inputs, training=False)  # MobileNetV2 backbone

x = bottleneck_block(x, filters=256, expansion=6, stride=1)

x = GlobalAveragePooling2D()(x)  
x = Dense(64, activation='relu6')(x) 
x = Dropout(0.2)(x) # Dropout 20% to prevent overfitting
predictions = Dense(20, activation='softmax')(x)  # 20 fruit classes

### Built Model MobileNetV2 with bottleneck_block and res_inception_module

In [ ]:
inputs = Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_model(inputs, training=False)  # MobileNetV2 backbone

x = bottleneck_block(x, filters=256, expansion=6, stride=1)

x = res_inception_module(x, 
                         filters_1x1=64, 
                         filters_3x3_reduce=96, filters_3x3=128,
                         filters_5x5_reduce=16, filters_5x5=32,
                         filters_pool=32)  

x = GlobalAveragePooling2D()(x)  
x = Dense(64, activation='relu6')(x) 
x = Dropout(0.2)(x) # Dropout 20% to prevent overfitting
predictions = Dense(20, activation='softmax')(x)  #20 fruit classes

### Complie mô hình

In [ ]:
model = Model(inputs, outputs=predictions)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

### Tóm tắt mô hình

In [ ]:
model.summary()

## Huấn luyện mô hình

In [ ]:
history = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS)

In [ ]:
model.save("MobileNet_Best_Model.h5")

In [ ]:
# Vẽ biểu đồ kết quả
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('MobileNetV2 - Fruits Classification')
plt.show()

### EVALUATING MODEL

In [ ]:
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from sklearn.metrics import f1_score
import numpy as np

# path
model_path = 'path.h5'
base_val = 'path_val'
base_test = 'path_test'
IMG_SIZE = 224
BATCH_SIZE = 32

# 1. load model
try:
    model = load_model(model_path)
    print("Model loaded successfully!")
except Exception as e:
    print(f"Error loading model: {str(e)}")
    exit()

# 2. size model
file_size_bytes = os.path.getsize(model_path)
file_size_mb = file_size_bytes / (1024 * 1024)
print(f"Model Size: {file_size_mb:.2f} MB ({file_size_bytes} bytes)")

datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

# 4. Evaluating on Val Set
print("\nEvaluating on Val Set:")
val_generator = datagen.flow_from_directory(
    base_val,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)
val_loss, val_accuracy = model.evaluate(val_generator)
print(f"Val Loss: {val_loss:.4f}")
print(f"Val Accuracy: {val_accuracy:.4f} ({val_accuracy * 100:.2f}%)")

# 5. Evaluating on Test Set
print("\nEvaluating on Test Set:")
test_generator = datagen.flow_from_directory(
    base_test,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f} ({test_accuracy * 100:.2f}%)")

# 6. F1-score
test_generator.reset()  
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = test_generator.classes
f1 = f1_score(y_true, y_pred_classes, average='weighted')
print(f"F1-Score (Test): {f1:.4f}")

# 7. Parameters
total_params = model.count_params()
print(f"Total Parameters: {total_params:,}")


